# Setting up FusionSC for your device

Fusionsc is extremely simple to start. Simply import the "fusionsc" package and the relevant machinery starts up in the background.

In [1]:
import fusionsc as fsc

## Device setup - J-TEXT

Currently, all used J-TEXT geometry is bundled inside the binary library and no actions should be neccessary. The coils and geometries are exposed in the `device.jtext` module.

## Device setup - Wendelstein 7-X

### On-site in the IPP network

In the IPP network, a set of database servers and compute servers is already set up.

In [1]:
from fusionsc.devices import w7x
w7x.connectIppSite()

### Off-site

Use an offline data file as outlined below.

### Pre-calculating fields for Biot-Savart calculation

A generally expensive calculation is the Biot-Savart rule to obtain the magnetic field from the coil geometries. For W7-X
, there is a high-level support to precompute the coil fields. These fields can then be saved and loaded later for
usage. When using precomputed coil fields, the offline data files are not required (unless required for other reasons, such
as geometry data or other field information). W7-X has special support to pre-compute all coils so that they can be re-used
later.

The precomputation is performed through the `devices.w7x.CoilPack` class, which can be used as a handle to override the coils
used in the generation of W7-X configurations.

In [1]:
from fusionsc.devices import w7x

# Obtain default coils
coils = w7x.cadCoils()

# Compute
coils = coils.computeFields(w7x.defaultGrid())

# Use in configuration
config1 = w7x.standard(coils = coils)
config2 = w7x.highIota(coils = coils)

The precalculated fields can be saved and loaded like other field or geometry classes.

In [ ]:
# Save
coils.save("coils.fsc")

# Restore
coils = w7x.CoilPack.load("coils.fsc")

## Specifying the sources for machine geometry information

### Using offline data files

FusionSC supports high-level descriptions for magnetic configurations and geometries. However, the data backing these fields
or components are often restricted by team membership limitations, and can therefore usually not be shipped inside FusionSC.

The `resolve` submodule drives the conversion of these machine-specific data nodes into generic nodes. This module
can be connected to various data sources, with the most important ones being *site-bound intranet databases* and *offline
data files*.

Opening an offline data file will result in the stored machine-specific information to be globally registered for the resolution
process.


In [ ]:
fsc.resolve.importOfflineData('yourDeviceFile.fsc')

### Using online databases

Facilities to use site-specific online databases are exposed through device-specific submodules (see below)

## (Re-)Configuring the local backend

While fusionsc is capable of connecting to remote servers for execution and data management, by default it uses a local backend started at import time. The backend is started with a default configuration (all services enabled, CPU execution, auto-detected no. of computation threads). Sometimes it is desirable to override this default behavior. You can use the `backends.reconfigureLocalBackend` function for this purpose.

*Note*: This function creates a **new** backend to be used by all future calls. The old backend will be closed once all references to it coming from previous pending calls are dropped. If the local backend has been exposed (e.g. as a network service or by storing the result of backends.activeBackend) it will not shut down until these external references are dropped as well.

In [2]:
# Get default configuration
config = fsc.service.LocalConfig.newMessage()
print(' --- Default configuration --- ')
print()
print(config)
print()

# Adjust configuration
config.cpuBackend.numThreads.fixed = 2

print(' --- Adjusted configuration --- ')
print()
print(config)
print()

# Apply adjusted configuration
fsc.backends.reconfigureLocalBackend(config)

 --- Default configuration --- 

preferredDeviceType: cpu
enableCompute: true
enableStorage: true
jobScheduler: system
flt:
  eventBuffer:
    minSize: 100
    maxSize: 2500
    targetTotalMb: 500
cpuBackend:
  numThreads: autoDetect

 --- Adjusted configuration --- 

preferredDeviceType: cpu
enableCompute: true
enableStorage: true
jobScheduler: system
flt:
  eventBuffer:
    minSize: 100
    maxSize: 2500
    targetTotalMb: 500
cpuBackend:
  numThreads:
    fixed: 2



## Using verbose logging

Sometimes it is desirable to enable the internal log output to see progress on the calculation. This can be achieved by setting the `FUSIONSC_VERBOSE` environment variable to any other value than `0`. This has to be done **before** importing the package.

In [1]:
import os
os.environ['FUSIONSC_VERBOSE'] = '1'

import fusionsc as fsc

When using the standalone server, verbose logging can be activated through the `--verbose` command-line switch.

In [5]:
from fusionsc.devices import jtext

## Other devices

Other devices do not have built-in support, but you can load your own geometries and coil filaments from numpy arrays. If you can provide us specifications and data sources for your device, we will gladly incoroporate a support library for your machine. Simpler geometries can also be bundled into the library itself.